目的：**SIFT特征提取和匹配具体步骤**

1.  生成高斯差分金字塔（DOG金字塔），尺度空间构建
2.  空间极值点检测（关键点的初步查探）
3.  稳定关键点的精确定位
4.  稳定关键点方向信息分配
5.  关键点描述
6.  特征点匹配


1. 导入常用库

In [4]:
import cv2
import numpy as np

2. drawMatchesKnn_cv2 作用：绘制关键点的匹配结果

In [5]:
def drawMatchesKnn_cv2(img1_gray,kp1,img2_gray,kp2,goodMatch):
    h1, w1 = img1_gray.shape[:2]
    h2, w2 = img2_gray.shape[:2]
 
    vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
    vis[:h1, :w1] = img1_gray
    vis[:h2, w1:w1 + w2] = img2_gray
 
    p1 = [kpp.queryIdx for kpp in goodMatch]
    p2 = [kpp.trainIdx for kpp in goodMatch]
 
    post1 = np.int32([kp1[pp].pt for pp in p1])
    post2 = np.int32([kp2[pp].pt for pp in p2]) + (w1, 0)
 
    for (x1, y1), (x2, y2) in zip(post1, post2):
        cv2.line(vis, (x1, y1), (x2, y2), (0,0,255))
 
    cv2.namedWindow("match",cv2.WINDOW_NORMAL)
    cv2.imshow("match", vis)
    


3. 找出相似度最高的前k个，在原图和对比图上绘制出来

In [6]:
img1_gray = cv2.imread("iphone1.png")
img2_gray = cv2.imread("iphone2.png")
 
#sift = cv2.SIFT()
sift =cv2.SIFT_create()  #新的调换方式  SIFT具体步骤1-4
# sift = cv2.xfeatures2d.SIFT_create() #旧的调换方式
#sift = cv2.SURF()
# 找出关键点并计算描述符
kp1, des1 = sift.detectAndCompute(img1_gray, None)
kp2, des2 = sift.detectAndCompute(img2_gray, None)
 
# BFmatcher with default parms
bf = cv2.BFMatcher(cv2.NORM_L2)
#opencv中knnMatch是一种蛮力匹配
#将待匹配图片的特征与目标图片中的全部特征全量遍历，找出相似度最高的前k个。
matches = bf.knnMatch(des1, des2, k = 2)

goodMatch = []
for m,n in matches:
    #两图的描述符之间的距离越小，说明两图越相似
    if m.distance < 0.50*n.distance:
        goodMatch.append(m)
 
drawMatchesKnn_cv2(img1_gray,kp1,img2_gray,kp2,goodMatch[:20])
 
cv2.waitKey(0)
cv2.destroyAllWindows()